In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore")

C:\Users\2202184\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [26]:
#Buscando os dados histórico do preço do ouro com a biblioteca yfinance.
AU_simbol = "GC=F"
FE_simbol = "TIO=F"
VALE_sombol = "VALE3.SA"
AU_ticker = yf.Ticker(AU_simbol)
FE_ticker = yf.Ticker(FE_simbol)
VALE_ticker = yf.Ticker(VALE_sombol)

In [27]:
#Dados de 5 anos
historical_dataAU = AU_ticker.history(period="5y")
historical_dataFE = FE_ticker.history(period="5y")
historical_dataVALE = VALE_ticker.history(period="5y")

In [30]:
dfAU = pd.DataFrame(historical_dataAU)
dfFE = pd.DataFrame(historical_dataFE)
dfVALE = pd.DataFrame(historical_dataVALE)

In [31]:
dfAU = dfAU.reset_index()
dfFE = dfFE.reset_index()
dfVALE = dfVALE.reset_index()

In [32]:
#Transformando a coluna data para o formato data hora
dfAU["Date"] = pd.to_datetime(dfAU["Date"], format="%Y-%m-%d")
dfFE["Date"] = pd.to_datetime(dfFE["Date"], format="%Y-%m-%d")
dfVALE["Date"] = pd.to_datetime(dfVALE["Date"], format="%Y-%m-%d")

In [19]:
figAU = px.line(
    dfAU,
    x="Date",
    y="Close",
    title="Evolução do Preço de Fechamento",
    labels={"Date": "Data", "Close": "Preço de Fechamento (USD)"}
)

# Personalizando o layout
figAU.update_layout(
    title_font=dict(size=20, family="Arial", color="darkblue"),
    xaxis_title="Data",
    yaxis_title="Preço do Ouro (USD)",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    plot_bgcolor="white",
    hovermode="x unified",
)

# Adicionando uma linha de referência horizontal (opcional)
figAU.add_hline(
    y=dfAU["Close"].mean(),
    line_dash="dot",
    annotation_text="Média",
    annotation_position="bottom right",
    line_color="red"
)

# Exibindo o gráfico
figAU.show()

In [21]:
figAU = px.line(
    dfFE,
    x="Date",
    y="Close",
    title="Evolução do Preço de Fechamento",
    labels={"Date": "Data", "Close": "Preço de Fechamento (USD)"}
)

# Personalizando o layout
figAU.update_layout(
    title_font=dict(size=20, family="Arial", color="darkblue"),
    xaxis_title="Data",
    yaxis_title="Preço do Ferro (USD)",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    plot_bgcolor="white",
    hovermode="x unified",
)

# Adicionando uma linha de referência horizontal (opcional)
figAU.add_hline(
    y=dfFE["Close"].mean(),
    line_dash="dot",
    annotation_text="Média",
    annotation_position="bottom right",
    line_color="red"
)

# Exibindo o gráfico
figAU.show()

In [35]:
figVALE = px.line(
    dfVALE,
    x="Date",
    y="Close",
    title="Evolução do Preço de Fechamento",
    labels={"Date": "Data", "Close": "Preço de Fechamento (USD)"}
)

# Personalizando o layout
figVALE.update_layout(
    title_font=dict(size=20, family="Arial", color="darkblue"),
    xaxis_title="Data",
    yaxis_title="Preço Ações VALE (USD)",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    plot_bgcolor="white",
    hovermode="x unified",
)

# Adicionando uma linha de referência horizontal (opcional)
figVALE.add_hline(
    y=dfVALE["Close"].mean(),
    line_dash="dot",
    annotation_text="Média",
    annotation_position="bottom right",
    line_color="red"
)

# Exibindo o gráfico
figVALE.show()

In [43]:
import numpy as np
import plotly.express as px

# Adicionando uma coluna numérica para representar o tempo
dfVALE["Date_Num"] = (dfVALE["Date"] - dfVALE["Date"].min()).dt.days

# Calculando a linha de tendência (regressão linear)
coeffs = np.polyfit(dfVALE["Date_Num"], dfVALE["Close"], 1)
trendline = coeffs[0] * dfVALE["Date_Num"] + coeffs[1]

# Criando o gráfico com a linha de tendência
figVALE = px.line(
    dfVALE,
    x="Date",
    y="Close",
    title="Evolução do Preço de Fechamento com Linha de Tendência",
    labels={"Date": "Data", "Close": "Preço de Fechamento (USD)"}
)

# Adicionando a linha de tendência
figVALE.add_scatter(
    x=dfVALE["Date"],
    y=trendline,
    mode="lines",
    name="Linha de Tendência",
    line=dict(color="orange", dash="dash")
)

# Personalizando o layout
figVALE.update_layout(
    title_font=dict(size=20, family="Arial", color="darkblue"),
    xaxis_title="Data",
    yaxis_title="Preço Ações VALE (USD)",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    plot_bgcolor="white",
    hovermode="x unified",
)

# Adicionando uma linha de referência horizontal (opcional)
figVALE.add_hline(
    y=dfVALE["Close"].mean(),
    line_dash="dot",
    annotation_text="Média",
    annotation_position="bottom right",
    line_color="red"
)

# Exibindo o gráfico
figVALE.show()


In [44]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Ajustando o modelo de suavização exponencial
model = ExponentialSmoothing(dfVALE["Close"], trend="add", seasonal=None, initialization_method="estimated")
model_fit = model.fit()

# Fazendo previsões
dfVALE["ETS_Prediction"] = model_fit.fittedvalues

# Adicionando a previsão ao gráfico
figVALE.add_scatter(
    x=dfVALE["Date"],
    y=dfVALE["ETS_Prediction"],
    mode="lines",
    name="Previsão (ETS)",
    line=dict(color="green", dash="dot")
)
figVALE.show()


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

# Pré-processando os dados
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dfVALE["Close"].values.reshape(-1, 1))

# Criando sequências para treinar o modelo
sequence_length = 60
X_train, y_train = [], []
for i in range(sequence_length, len(scaled_data)):
    X_train.append(scaled_data[i-sequence_length:i, 0])
    y_train.append(scaled_data[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Construindo o modelo LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))

# Compilando e treinando o modelo
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, epochs=200, batch_size=60, verbose=1)

# Prevendo os próximos 30 dias
last_sequence = scaled_data[-sequence_length:]  # Última sequência para iniciar a previsão
predictions = []
for _ in range(30):
    pred = model.predict(last_sequence.reshape(1, sequence_length, 1))
    predictions.append(pred[0, 0])
    last_sequence = np.append(last_sequence[1:], pred, axis=0)  # Atualizando a sequência

# Revertendo o escalonamento para valores originais
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

# Criando um DataFrame para os próximos 30 dias
future_dates = pd.date_range(dfVALE["Date"].iloc[-1] + pd.Timedelta(days=1), periods=30)
df_future = pd.DataFrame({"Date": future_dates, "Close": predictions.flatten()})

# Plotando o gráfico
figVALE = px.line(
    dfVALE,
    x="Date",
    y="Close",
    title="Evolução do Preço de Fechamento com Previsão LSTM",
    labels={"Date": "Data", "Close": "Preço de Fechamento (USD)"}
)

# Adicionando a previsão LSTM ao gráfico
figVALE.add_scatter(
    x=df_future["Date"],
    y=df_future["Close"],
    mode="lines",
    name="Previsão (LSTM - 30 dias)",
    line=dict(color="orange", dash="dot")
)

# Adicionando a média ao gráfico
figVALE.add_hline(
    y=dfVALE["Close"].mean(),
    line_dash="dot",
    annotation_text="Média",
    annotation_position="bottom right",
    line_color="red"
)

# Personalizando o layout
figVALE.update_layout(
    title_font=dict(size=20, family="Arial", color="darkblue"),
    xaxis_title="Data",
    yaxis_title="Preço Ações VALE (USD)",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    plot_bgcolor="white",
    hovermode="x unified",
)

# Exibindo o gráfico
figVALE.show()


Epoch 1/200
20/20 [==============================] - 8s 133ms/step - loss: 0.0519
Epoch 2/200
20/20 [==============================] - 2s 123ms/step - loss: 0.0046
Epoch 3/200
20/20 [==============================] - 3s 124ms/step - loss: 0.0029
Epoch 4/200
20/20 [==============================] - 2s 121ms/step - loss: 0.0025
Epoch 5/200
20/20 [==============================] - 3s 136ms/step - loss: 0.0022
Epoch 6/200
20/20 [==============================] - 3s 130ms/step - loss: 0.0020
Epoch 7/200
20/20 [==============================] - 3s 137ms/step - loss: 0.0019
Epoch 8/200
20/20 [==============================] - 3s 128ms/step - loss: 0.0017
Epoch 9/200
20/20 [==============================] - 2s 123ms/step - loss: 0.0016
Epoch 10/200
20/20 [==============================] - 2s 117ms/step - loss: 0.0016
Epoch 11/200
20/20 [==============================] - 3s 133ms/step - loss: 0.0016
Epoch 12/200
20/20 [==============================] - 3s 138ms/step - loss: 0.0016
Epoch 13/200


Epoch 97/200
20/20 [==============================] - 2s 107ms/step - loss: 4.6625e-04
Epoch 98/200
20/20 [==============================] - 2s 109ms/step - loss: 4.8759e-04
Epoch 99/200
20/20 [==============================] - 2s 110ms/step - loss: 5.4131e-04
Epoch 100/200
20/20 [==============================] - 2s 110ms/step - loss: 4.8490e-04
Epoch 101/200
20/20 [==============================] - 2s 112ms/step - loss: 4.5649e-04
Epoch 102/200
20/20 [==============================] - 2s 120ms/step - loss: 4.9387e-04
Epoch 103/200
20/20 [==============================] - 2s 110ms/step - loss: 4.9418e-04
Epoch 104/200
20/20 [==============================] - 2s 119ms/step - loss: 4.7798e-04
Epoch 105/200
20/20 [==============================] - 2s 121ms/step - loss: 5.0633e-04
Epoch 106/200
20/20 [==============================] - 2s 112ms/step - loss: 5.9350e-04
Epoch 107/200
20/20 [==============================] - 2s 111ms/step - loss: 4.9581e-04
Epoch 108/200
20/20 [==============

In [ ]:
df_predictions = pd.DataFrame({
    "Date": future_dates,            # Datas futuras geradas
    "Predicted_Close": predictions.flatten()  # Valores previstos pelo modelo LSTM
})



In [ ]:
df_predictions.head()

In [ ]:
df_predictions.describe()